<a href="https://colab.research.google.com/github/perchedinthedark/formula1_predictions/blob/main/imola.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install fastf1 pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 0.7.3 requires websockets>=14.0, but you have websockets 13.1 which is incompatible.


In [3]:
import fastf1
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

# Cache directory set-up
cache_dir = "./cache"
os.makedirs(cache_dir, exist_ok=True)
fastf1.Cache.enable_cache(cache_dir)

# Load FastF1 2024 Australian GP race session
session_2024 = fastf1.get_session(2024, "Imola", "R")
session_2024.load()

# Extract lap times
laps_2024 = session_2024.laps[["Driver", "LapTime", "Stint", "LapNumber"]].copy()
laps_2024.dropna(subset=["LapTime"], inplace=True)
laps_2024["LapTime (s)"] = laps_2024["LapTime"].dt.total_seconds()


# 2025 Qualifying Data Imola GP
qualifying_2025 = pd.DataFrame({
    "Driver": ["Oscar Piastri", "Max Verstappen", "George Russell", "Lando Norris", "Fernando Alonso",
               "Carlos Sainz Jr.", "Alexander Albon", "Lance Stroll", "Isack Hadjar", "Pierre Gasly",
               "Charles Leclerc", "Lewis Hamilton", "Andrea Kimi Antonelli", "Gabriel Bortoleto", "Franco Colapinto",
               "Liam Lawson", "Nico Hülkenberg", "Esteban Ocon", "Oliver Bearman", "Yuki Tsunoda"],
    "QualifyingTime (s)": [74.670, 74.704, 74.807, 74.962, 75.431,
                           75.198, 75.473, 75.497, 75.510, 75.505,
                           75.604, 75.765, 75.772, 76.260, 76.256,
                           76.379, 76.518, 76.613, 76.918, 0]
})

# Map full names to FastF1 3-letter codes
driver_mapping = {
    "Oscar Piastri": "PIA", "George Russell": "RUS", "Lando Norris": "NOR", "Max Verstappen": "VER",
    "Lewis Hamilton": "HAM", "Charles Leclerc": "LEC", "Isack Hadjar": "HAD", "Andrea Kimi Antonelli": "ANT",
    "Yuki Tsunoda": "TSU", "Alexander Albon": "ALB", "Esteban Ocon": "OCO", "Nico Hülkenberg": "HUL",
    "Fernando Alonso": "ALO", "Lance Stroll": "STR", "Carlos Sainz Jr.": "SAI", "Pierre Gasly": "GAS",
    "Oliver Bearman": "BEA", "Franco Colapinto": "COL", "Gabriel Bortoleto": "BOR", "Liam Lawson": "LAW"
}

# 2025 Driver Standings (Example values - replace with real standings if available)
driver_standings_2025 = {
    "Oscar Piastri": 1, "Max Verstappen": 3, "George Russell": 4, "Lewis Hamilton": 7, "Lando Norris": 2,
    "Charles Leclerc": 5, "Isack Hadjar": 16, "Andrea Kimi Antonelli": 6, "Yuki Tsunoda": 11,
    "Alexander Albon": 8, "Esteban Ocon": 9, "Nico Hülkenberg": 14, "Fernando Alonso": 17,
    "Lance Stroll": 10, "Carlos Sainz Jr.": 13, "Pierre Gasly": 12, "Oliver Bearman": 15,
    "Franco Colapinto": 20, "Gabriel Bortoleto": 19, "Liam Lawson": 18
}

# Invert the standings (1st → highest value, 20th → lowest)
max_rank = max(driver_standings_2025.values())  # Gets the worst rank (e.g., 20)
driver_standings_2025_inverted = {
    driver: max_rank + 1 - rank
    for driver, rank in driver_standings_2025.items()
}

# Map inverted standings to the DataFrame
qualifying_2025["DriverStandings2025"] = qualifying_2025["Driver"].map(driver_standings_2025_inverted)

# Optional: Add driver codes if needed
qualifying_2025["DriverCode"] = qualifying_2025["Driver"].map(driver_mapping)

# Merge 2025 Qualifying Data with 2024 Race Data
merged_data = qualifying_2025.merge(laps_2024, left_on="DriverCode", right_on="Driver")


# Use both Qualifying time and driver standings as features
# X = merged_data[["QualifyingTime (s)", "DriverStandings2025", "DegradationRate"]]
X = merged_data[["QualifyingTime (s)", "DriverStandings2025"]]
y = merged_data["LapTime (s)"]

if X.shape[0] == 0:
    raise ValueError("Dataset is empty after preprocessing. Check data sources!")

# Train Gradient Boosting Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=39)
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=39)
model.fit(X_train, y_train)

# Predict using 2025 qualifying times
# predicted_lap_times = model.predict(qualifying_2025[["QualifyingTime (s)", "DriverStandings2025", "DegradationRate"]])
predicted_lap_times = model.predict(qualifying_2025[["QualifyingTime (s)", "DriverStandings2025"]])
qualifying_2025["PredictedRaceTime (s)"] = predicted_lap_times

# Rank drivers by predicted race time
qualifying_2025 = qualifying_2025.sort_values(by="PredictedRaceTime (s)")

# Print final predictions
print("\n🏁 Predicted 2025 Imola GP Winner with no Change in ML Model🏁\n")
print(qualifying_2025[["Driver", "PredictedRaceTime (s)"]])

# Evaluate Model
y_pred = model.predict(X_test)
print(f"\n🔍 Model Error (MAE): {mean_absolute_error(y_test, y_pred):.2f} seconds")

core           INFO 	Loading data for Emilia Romagna Grand Prix - Race [v3.5.3]
INFO:fastf1.fastf1.core:Loading data for Emilia Romagna Grand Prix - Race [v3.5.3]
req            INFO 	No cached data found for session_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
INFO:fastf1.api:Fetching session info data...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
INFO:fastf1.api:Fetching driver list...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
INFO:fastf1.fastf1.req:No cached data found for se


🏁 Predicted 2025 Imola GP Winner with no Change in ML Model🏁

                   Driver  PredictedRaceTime (s)
10        Charles Leclerc              81.124294
3            Lando Norris              81.423208
1          Max Verstappen              81.445264
11         Lewis Hamilton              81.631226
12  Andrea Kimi Antonelli              81.631226
0           Oscar Piastri              81.632398
5        Carlos Sainz Jr.              81.765878
2          George Russell              82.349001
7            Lance Stroll              82.730605
9            Pierre Gasly              82.832476
19           Yuki Tsunoda              82.930353
16        Nico Hülkenberg              83.236615
18         Oliver Bearman              83.290386
15            Liam Lawson              83.527918
14       Franco Colapinto              83.527918
13      Gabriel Bortoleto              83.527918
8            Isack Hadjar              83.575389
17           Esteban Ocon              83.623831
6     